In [1]:
import pandas as pd
import numpy as np
import spacy
from deep_translator import GoogleTranslator
import re

In [2]:
df = pd.read_excel('Survey_SleepHabits.xlsx')

In [3]:
df.columns

Index(['طابع زمني', '‎1. ‭What is your age ?', '‎2. ‭What is your gender ?',
       '‎3. ‭What is your occupation ?', '‎4. ‭What is your marital status ?',
       '‎5. ‭How many hours do you sleep on average per night ?',
       '‎6. ‭At what time do you usually go to bed ?',
       '‎7. ‭How would you rate the quality of your sleep ?',
       '‎8. ‭What do you feel on average when you wake up ?',
       '‎9. ‭How often do you feel sleepy during the daytime ?',
       '‎10. ‭Are there any factors that interrupt your sleep or make you unable sleep ? (e.g., sound, light, room temperature, etc...)',
       '‎11. ‭If so, explain what factor(s) and how it affects your sleep ?',
       '‎12. ‭Is your sleep often interrupted resulting in waking up in the middle of the night ?',
       '‎13. ‭If so, what do you do to go back to sleep ?',
       '‎14. ‭What activities do you usually do before bedtime ? (select all that apply)',
       '‎15. ‭What do you think about these activities' effect, is 

In [4]:
Data = df.iloc[:,[11,13,15,17,19,20]]

In [5]:
Data.columns

Index(['‎11. ‭If so, explain what factor(s) and how it affects your sleep ?',
       '‎13. ‭If so, what do you do to go back to sleep ?',
       '‎15. ‭What do you think about these activities' effect, is it positive or negative on your sleep ?',
       '‎17. ‭If you have difficulty falling asleep, what do you do to fall asleep ? (do you force yourself to sleep, listen to music/podcast or do other activities). Please explain',
       '‎19. ‭If you have any sleep disorders, what do you do to overcome it and to cope with it ?',
       '‎20. ‭If you use electronic devices (e.g., phone, TV, laptop, etc...)\n? how many hours do you spend using them'],
      dtype='object')

In [6]:
def translate_text(text, target_language='en'):
    if isinstance(text, str) and text.strip():
        try:
            translator = GoogleTranslator(source='auto', target=target_language)
            return translator.translate(text)
        except Exception as e:
            print(f"Translation error for '{text}': {e}")
            return text
    return "no respnd"

def translate_dataframe(df, target_language='en'):
    translated_df = df.copy()
    
    for col in df.columns:
        translated_column = df[col].astype(str).apply(lambda x: translate_text(x, target_language))
        translated_df[f'{col}_'] = translated_column

    return translated_df

In [7]:
translated_df = translate_dataframe(Data, target_language='en')


In [8]:
data2 = translated_df.iloc[:,6:]

In [9]:
data2.head()

,"‎11. ‭If so, explain what factor(s) and how it affects your sleep ?_","‎13. ‭If so, what do you do to go back to sleep ?_","‎15. ‭What do you think about these activities' effect, is it positive or negative on your sleep ?_","‎17. ‭If you have difficulty falling asleep, what do you do to fall asleep ? (do you force yourself to sleep, listen to music/podcast or do other activities). Please explain_","‎19. ‭If you have any sleep disorders, what do you do to overcome it and to cope with it ?_","‎20. ‭If you use electronic devices (e.g., phone, TV, laptop, etc...)\n? how many hours do you spend using them_"
0,Street noise,Lay down for some time,Negative,Force myself to sleep,into,About 5 to 6
1,Light disturb my sleep,I close my eyes,Negative,Listening to quran recitation,None,into
2,into,into,into,into,into,into
3,Light,into,Negative,into,into,More than 5 hours daily
4,light make me take more time to fall in sleep ...,into,morning sun good for sleep earily also close t...,force myself to sleep,into,9


In [10]:
colum1 = data2.iloc[:,5]

In [11]:
from translate import Translator
from langdetect import detect, LangDetectException

translator = Translator(from_lang='ar', to_lang="en")

def translate_if_needed(text):
    if pd.isna(text) or not str(text).strip() or str(text).strip() == ".":
        return "no respnd"
    text_str = str(text).strip()
    try:
        language = detect(text_str)
        if language == 'ar':
            return translator.translate(text_str)
        else:
            return text_str
    except LangDetectException:
        return text_str
    except Exception as e:
        print(f"Error during translation for '{text}': {e}")
        return text_str

def translate_first_n_columns_inplace(df, n=5):

    columns_to_translate = df.columns[:min(n, len(df.columns))].tolist()
    for col in columns_to_translate:
        translated_values = []
        for value in df[col]:
            translation = translate_if_needed(value)
            translated_values.append(translation)
        df[col] = translated_values  # Replace the original column
    return df


# Translate the first 5 columns and save to a new DataFrame
translated_df = translate_first_n_columns_inplace(data2, n=5)



Error during translation for 'مش بعرف انام لو الجو حر باخد وقت كبير ، النور برضه بيبقي مؤثر ومش بعرف انام منه': Expecting value: line 1 column 1 (char 0)


In [12]:
translated_df

,"‎11. ‭If so, explain what factor(s) and how it affects your sleep ?_","‎13. ‭If so, what do you do to go back to sleep ?_","‎15. ‭What do you think about these activities' effect, is it positive or negative on your sleep ?_","‎17. ‭If you have difficulty falling asleep, what do you do to fall asleep ? (do you force yourself to sleep, listen to music/podcast or do other activities). Please explain_","‎19. ‭If you have any sleep disorders, what do you do to overcome it and to cope with it ?_","‎20. ‭If you use electronic devices (e.g., phone, TV, laptop, etc...)\n? how many hours do you spend using them_"
0,Street noise,Lay down for some time,Negative,Force myself to sleep,into,About 5 to 6
1,Light disturb my sleep,I close my eyes,Negative,Listening to quran recitation,no respnd,into
2,into,into,into,into,into,into
3,Light,into,Negative,into,into,More than 5 hours daily
4,light make me take more time to fall in sleep ...,into,morning sun good for sleep earily also close t...,force myself to sleep,into,9
...,...,...,...,...,...,...
192,into,into,into,into,into,into
193,others not sleeping early,listen to brown noise,negative,prepare my environment for sleep,into,into
194,into,into,into,into,into,into
195,Annoying snoring make me unable to sleep:(,into,I think using phone or any other electronic de...,"Nothing, just wait untill sleeping...",into,Most of the day hours using laptop or phone


In [13]:
firstColum = translated_df.iloc[:,0].values

In [14]:
nlp = spacy.load("en_core_web_lg")


In [ ]:
question_1 = """"
cannot sleep
disturb my sleep
take more time to fall in sleep
unable to sleep
affects the duration of my sleep
feel a headache
start my day upset
cannot sleep well
bad dreams
affected if there is a sound or light
wake up to solve this problem and return to sleep
do not remember that
disrupt my sleep
Stress
anxiety
overthinking
not able to sleep
sleep go from me
cannot sleep
make me wake up
wake up from the slightest noise
not sleeping
Make me tired along my sleep
alot of thinking make me take long time to fall a sleep
feel uncomfortable
can sleep well in total silence only
make me walk up
make me uncomfortable
don't sleep well
Thinking about future
can't sleep when the weather is hot
Stress overthing
Loud Noise, i hate noise
Fear of wetting my bed
too hot weather
fear of death while iam sleeping
wake up and cannot sleep again
any sound and light can wake me up
room temperature
I wake up during sleep
Light and some times sound
Any body speak near of me or if I have problem I think about it
do not know how to sleep if it is free to take a lot of time
light also remains effective and I do not know how to sleep from it
My mind is constantly bugging me and telling me to wake up
Sound and light make me feel angry
Any sound , motion or light will wake me up sadly i am a very restless sleeper
Thinking
children voice , bad effect
Thinking,bad effect
I can not sleep with sound,light
Sounds and lights make my sleep so hard
This factors are annoying
The noise
Noise , caffeine abuse
make me disturbed and don't sleep well
A disturbed night’s sleep can lead to sleepiness and irritability
Stress
Anxiety, stress and noise
High temperature in summer make it hard to sleep
Too much light and high sound
high sound
Annoying snoring make me unable to sleep
High sounds and noisy
Maybe light is the only thing that makes me not able to sleep
Annoying snoring make me unable to sleep:(
Physical pain
Stress
Mental health issues
Snoring
Sleep disorders
Light
Jet lag
Medications
Sleep environment
Sleep schedule
Caffeine & alcohol
Certain foods
Internal Factors Affecting Sleep Quality
"""

editedFirstColum = []
doc2 = nlp(question_1)
for i in range(len(firstColum)):
    row = []
    # print(firstColum[i])
    for token in nlp(firstColum[i]):
        if token.lemma_ in row:
            continue
        # print(token.text, token.pos_, token.dep_, token.lemma_)
        if token.pos_ == 'NOUN' or token.dep_ == 'conj'  or token.dep_ == 'compound' :
                row.append(token.lemma_)
    # print(row)
    # print("__________________________")
    editedFirstColum.append(row)

In [ ]:
for i in range(len(editedFirstColum)):
    print(editedFirstColum[i])
    print(firstColum[i])
    print("--------------------------------------------------")

In [17]:
FinalFirstColum = []

In [18]:
for i in range(len(editedFirstColum)):
    row = []
    for element in editedFirstColum[i]:
        token = nlp(element)
        if token.similarity(doc2) > 0.5:
            row.append(token.text)
    FinalFirstColum.append(row)

/tmp/ipykernel_4585/3887156030.py:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if token.similarity(doc2) > 0.5:


In [ ]:
for i in range(len(FinalFirstColum)):
    # print(FinalFirstColum[i])
    print(editedFirstColum[i])
    # print("--------------------------------------------------")

In [92]:
mapping = {
    "noise": "sound",
    "noisy": "sound",
    "voice": "sound",
    "sounds": "sound",
    "light": "light",
    "lights": "light",
    "temperature": "temperature",
    "stress": "stress",
    "anxiety": "stress",
    "thinking": "overthinking",
    "overthinking": "overthinking",
    "weather": "weather",
    "hot": "weather",
    "summer": "weather",
    "factors": "sound and light",
    "room": "sleep environment",
    "children": "sound",
    "snoring": "sound",
    "child": "sound",
    "work": "stress",
    "job": "stress",
    "study": "stress",
    "family": "disturbance",
    "siblings": "disturbance",
    "fear": "fear",
    "bed": "sleep environment",
    "bedwetting": "fear",
    "music": "sound",
    "time": "lack of sleep",
    "alarm": "lack of sleep",
    "motion": "motion",
    "traffic": "sound",
    "crowd": "sound",
    "relations": "disturbance",
    "friend": "disturbance",
    "future": "overthinking",
    "mind": "overthinking",
    "telling": "overthinking",
    "opponent": "overthinking",
    "game": "overthinking",
    "health": "disturbance",
    "caffeine": "caffeine",
}

# Function to normalize terms in the array
def normalize_terms(array, mapping):
    noRespondsIndex = []
    normalized_array = []
    for i,row in enumerate(array):
        normalized_row = []
        for word in row:
            # Normalize the word using the mapping dictionary
            normalized_word = mapping.get(word.lower())
            if normalized_word:  # Only add words that exist in the mapping
                normalized_row.append(normalized_word)
        if not normalized_row and not word:
            noRespondsIndex.append(i+1)
        # Remove duplicates and sort the row
        if not normalized_row:
            normalized_array.append(["no respond"])
            continue
        normalized_array.append(sorted(set(normalized_row)))
    return normalized_array,noRespondsIndex



# Normalize the input array
normalized_array,index1 = normalize_terms(editedFirstColum, mapping)

# Print the normalized output
for row in normalized_array:
    print(row)
for i in index1:
    print(i)

['sound']
['no respond']
['no respond']
['light']
['lack of sleep', 'light']
['no respond']
['lack of sleep', 'sound', 'temperature']
['lack of sleep', 'sound']
['light']
['light']
['overthinking', 'stress']
['no respond']
['no respond']
['no respond']
['sound and light']
['no respond']
['no respond']
['no respond']
['no respond']
['no respond']
['light']
['no respond']
['light']
['no respond']
['no respond']
['no respond']
['no respond']
['sound']
['no respond']
['light', 'sound']
['no respond']
['no respond']
['no respond']
['no respond']
['sleep environment', 'temperature', 'weather']
['light', 'sound']
['no respond']
['no respond']
['light', 'sound', 'sound and light']
['no respond']
['no respond']
['no respond']
['no respond']
['no respond']
['lack of sleep', 'overthinking']
['no respond']
['sound']
['no respond']
['no respond']
['sound', 'temperature']
['sound']
['no respond']
['sleep environment', 'temperature']
['light', 'temperature']
['no respond']
['no respond']
['light']
['

In [81]:
for i in range(len(normalized_array)):
    print(normalized_array[i])
    print(firstColum[i])

['sound']
Street noise
['no respond']
Light disturb my sleep
['no respond']
into
['light']
Light
['light', 'time']
light make me take more time to fall in sleep same for sound too
['no respond']
into
['sound', 'temperature', 'time']
Of course the sounds make me unable to sleep and this affects the duration of my sleep and the time I wake up, I feel a headache and start my day upset and also if the temperature is high I cannot sleep well
['sound', 'time']
Time I sleep at ,noise,bad dreams
['light', 'problem']
I affected if there is a sound or light.  I wake up to solve this problem and return to sleep.  usually I do not remeber that
['light']
Loud sound and light disrupt my sleep
['stress', 'thinking']
Stress and anxiety and overthinking
['no respond']
into
['no respond']
into
['no respond']
into
['factors']
There's no factors
['no respond']
into
['no respond']
Alarms
['no respond']
into
['time']
I need to listen to musik or watch somthing to sleep
['no respond']
into
['light']
Light an

In [45]:
SecondColum = translated_df.iloc[:,1].values

In [ ]:
question_2 = """"
If you wake up in the middle of the night, try these strategies to fall back asleep:

Block out loud sounds with earplugs, white noise, or a fan
Get out of bed and do something relaxing for 15 minutes
Avoid screens and bright lights, which can suppress melatonin production
Meditate or try breathing exercises, such as the 4-7-8 technique
Relax your muscles with a full-body scan or listening to relaxing music
Focus on something boring, like counting sheep or reading an uninteresting article
Try sleep apps with relaxing stories, music, or sounds or quran
"""
editedSecColum = []
doc3 = nlp(question_2)
for i in range(len(SecondColum)):
    row = []
    print(SecondColum[i])
    for token in nlp(SecondColum[i]):
        # print(token.text, token.pos_, token.dep_, token.lemma_)
        if token.lemma_ in row:
            continue
        if token.pos_ == 'VERB' or token.dep_ == 'compound' or token.pos_ == 'NOUN':
                row.append(token.lemma_)
        elif token.similarity(doc3) > 0.78:
            row.append(token.text)
    editedSecColum.append(row)
    print(row)
    print("__________________________")

In [ ]:
# Define a mapping dictionary for relevant actions or phrases
mapping = {
    "lay": "force myself to sleep",
    "close": "force myself to sleep",
    "organize": "organize sleeping place",
    "ask": "ask for forgiveness",
    "read": "read",
    "wait": "force myself to sleep",
    "hold": "avoid phone",
    "try": "try to sleep",
    "go": "force myself to sleep",
    "need": "force myself to sleep",
    "get": "force myself to sleep",
    "eat": "eat",
    "drink": "drink water",
    "use": "use phone",
    "wake": "do nothing",
    "sit": "sit for a while",
    "watch": "watch something",
    "pray": "pray",
    "return": "return to bed",
    "stay": "stay in bed",
    "think": "think about dreams",
    "distract": "distract thoughts",
    "listen": "listen to sounds",
    "finish": "finish tasks",
    "take": "take a breath",
    "calm": "calm myself",
    "play": "play games",
    "change": "change clothes",
    "direction": "sleep direction",
    "video": "watch videos",
    "mind": "mind games",
    "matter": "focus on what matters",
    "win": "win games",
    "cause": "cause relaxation",
    "mobil": "use phone",
    "youtube": "watch videos",
    "quran": "listen to Quran",
    "white": "listen to white noise",
    "noise": "listen to white noise",
    "nothing": "do nothing",
    "automatic": "automatic action",
    "dream": "think about dreams",
    "breathe": "take a breath",
    "calm": "calm myself",
}

# Function to extract relevant actions or phrases
def extract_relevant_phrases(sentences, mapping):
    extracted_phrases = []
    for sentence in sentences:
        relevant_phrases = []
        for word in sentence.split():  # Split sentence into words
            normalized_word = mapping.get(word.lower())
            if normalized_word:  # Check if the word is in the mapping
                relevant_phrases.append(normalized_word)
        if relevant_phrases:
            extracted_phrases.append((sorted(set(relevant_phrases))))  # Remove duplicates and join
        else:
            extracted_phrases.append("no respond")  # If no relevant phrases found, add "no respond"
    return extracted_phrases


extracted_phrases = extract_relevant_phrases(SecondColum, mapping)

# Print the extracted phrases
for i, phrase in enumerate(extracted_phrases):
    print(f"Record {i+1}: {phrase}")

In [24]:
ThirdColum = translated_df.iloc[:,2].values


In [ ]:
# Define a mapping dictionary for normalizing responses
mapping = {
    "negative": "negative",
    "negatively": "negative",
    "negatively affect": "negative",
    "negative effect": "negative",
    "negative,": "negative",
    "negtaive": "negative",
    "negtive": "negative",
    "positive": "positive",
    "positives": "positive",
    "positive effect": "positive",
    "very positive": "positive",
    "maybe negative": "negative",
    "may be negative": "negative",
    "might be negative": "negative",
    "both": "both positive and negative",
    "some are positive and some are negative": "both positive and negative",
    "sometimes positive and sometimes negative": "both positive and negative",
    "in between": "both positive and negative",
    "no effect": "no effect",
    "not affect": "no effect",
    "doesn’t affect me": "no effect",
    "no respnd": "no response",
    "don't know": "no response",
    "i don't know": "no response",
    "no": "negative",
    "yes": "positive",
}

# Function to normalize responses
def normalize_responses(column, mapping):
    normalized_responses = []
    for response in column:
        response_lower = str(response).lower().strip()
        normalized_response = mapping.get(response_lower, "no response")  # Default to "other" if not in mapping
        normalized_responses.append(normalized_response)
    return normalized_responses



# Normalize responses
normalized_responses3 = normalize_responses(ThirdColum, mapping)

# Print normalized responses
for i, response in enumerate(normalized_responses3):
    print(f"Record {i+1}: {response}")

In [79]:
normalized_responses3

['negative',
 'negative',
 'no respond',
 'negative',
 'no respond',
 'no respond',
 'no respond',
 'negative',
 'positive',
 'no respond',
 'negative',
 'negative',
 'no respond',
 'no respond',
 'no respond',
 'negative',
 'negative',
 'no respond',
 'no respond',
 'no respond',
 'negative',
 'negative',
 'negative',
 'no respond',
 'negative',
 'no respond',
 'no respond',
 'negative',
 'negative',
 'no response',
 'no respond',
 'no respond',
 'no respond',
 'no respond',
 'no respond',
 'no respond',
 'negative',
 'negative',
 'no respond',
 'no respond',
 'no respond',
 'no respond',
 'negative',
 'no respond',
 'no respond',
 'no respond',
 'no respond',
 'positive',
 'negative',
 'no respond',
 'positive',
 'no respond',
 'no respond',
 'both positive and negative',
 'no respond',
 'no respond',
 'negative',
 'negative',
 'negative',
 'no respond',
 'no respond',
 'no respond',
 'no respond',
 'positive',
 'negative',
 'no response',
 'no respond',
 'no respond',
 'no respond',

In [26]:
FourthColum = translated_df.iloc[:,3].values


In [ ]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

# Define the pattern: VERB + any tokens (1 or more) + NOUN
pattern = [
    {'POS': 'VERB'},  # Match a verb
    {'OP': '+'},      # Match one or more tokens (any type)
    {'POS': 'NOUN'}   # Match a noun
]

# Add the pattern to the matcher
matcher.add('VERB_NOUN_PATTERN', [pattern], greedy='LONGEST')

editedFourthColum = []

for i in range(len(FourthColum)):
    row = []
    print(FourthColum[i])
    doc = nlp(FourthColum[i])
    matches = matcher(doc)
    for token in nlp(FourthColum[i]):
        # print(token.text, token.pos_, token.dep_)
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN' or token.pos_== 'VERB' :
                row.append(token.text)

    # Extract only the first match per statement
    if matches:
        match_id, start, end = matches[0]  # Get the first match
        row.append(doc[start:end].text)  # Extract the matched text

    editedFourthColum.append(row)
    print(row)
    print("--------------------------------------------------")

Force myself to sleep
['Force', 'sleep']
--------------------------------------------------
Listening to quran recitation
['Listening', 'quran', 'recitation', 'Listening to quran recitation']
--------------------------------------------------
into
[]
--------------------------------------------------
into
[]
--------------------------------------------------
force myself to sleep
['force', 'sleep']
--------------------------------------------------
into
[]
--------------------------------------------------
into
[]
--------------------------------------------------
To  stay calm
['stay']
--------------------------------------------------
listening to quran or podcast
['listening', 'quran', 'podcast', 'listening to quran or podcast']
--------------------------------------------------
Listen to podcasts or scroll mindlessly
['Listen', 'podcasts', 'scroll', 'Listen to podcasts']
--------------------------------------------------
into
[]
--------------------------------------------------
in

In [ ]:
for row in editedFourthColum:
    print(row)

In [71]:
# Updated mapping dictionary with categories and keywords
mapping = {
    # Force Sleep
    "force": "force myself to sleep",
    "forcing": "force myself to sleep",
    "forced": "force myself to sleep",
    "sleel": "force myself to sleep",  # Assuming typo for "sleep"
    "self": "force myself to sleep",
    "way": "force myself to sleep",
    "success": "force myself to sleep",
    "learned": "force myself to sleep",

    # Listen to Quran
    "quran": "listen to Quran",
    "recitation": "listen to Quran",
    "quraan": "listen to Quran",
    "qur’an": "listen to Quran",
    "qur'an": "listen to Quran",

    # Listen to Music/Podcast
    "music": "listen to music",
    "podcast": "listen to podcast",
    "activities": "do other activities",
    "youtube": "watch YouTube",
    "videos": "watch videos",

    # Watch Videos/YouTube
    "watch": "watch videos",
    "watching": "watch videos",
    "episode": "watch videos",
    "tv": "watch videos",
    "videogame": "play video games",

    # Read/Story
    "read": "read",
    "story": "read a story",
    "asmr": "listen to ASMR",

    # Relaxation Techniques
    "relax": "relax",
    "unwind": "relax",
    "meditate": "meditate",
    "mindfulness": "relax",
    "diet": "relax",
    "exercise": "do physical activity",
    "environment": "prepare sleep environment",
    "breath": "focus on breathing",
    "breathe": "focus on breathing",

    # Physical Activity
    "activity": "do physical activity",
    "shower": "take a shower",
    "walking": "walk",
    "banana": "eat banana",

    # Prepare Sleep Environment
    "prepare": "prepare sleep environment",
    "covering": "prepare sleep environment",

    # Distract Mind
    "think": "distract mind",
    "imagine": "distract mind",
    "count": "count numbers",
    "numbers": "count numbers",

    # Use Phone/Media
    "phone": "use phone",
    "mobile": "use phone",
    "media": "use phone",
    "force": "force myself to sleep",
"music": "listen to music",
"podcast": "listen to podcast",
"quran": "listen to Quran",
"recitation": "listen to Quran",
"activities": "do other activities",
"watch": "watch videos",
"youtube": "watch YouTube",
"read": "read",
"story": "read a story",
"pray": "pray",
"meditate": "meditate",
"relax": "relax",
"shower": "take a shower",
"walk": "walk",
"count": "count numbers",
"imagine": "imagine",
"prepare": "prepare sleep environment",
"environment": "prepare sleep environment",
"wait": "wait",
"eat": "eat",
"drink": "drink water",
"breathe": "focus on breathing",
"mind": "clear my mind",
"stay": "stay in bed",
"phone": "use phone",
"mobile": "use phone",
"asmr": "listen to ASMR",
"light": "adjust light",
"melatonin": "take melatonin",
"exercise": "do physical activity",
"banana": "eat banana",
"videogame": "play video games",
"nothing": "do nothing",

}

# Function to normalize responses
def clean_none_values(records):
    cleaned_records = []
    for record in records:
        if isinstance(record, set):
            cleaned_record = {item for item in record if item is not None}
            cleaned_records.append(cleaned_record if cleaned_record else ['no respond'])
        elif isinstance(record, list):
            cleaned_records.append(record)
        else:
            cleaned_records.append(record)
    return cleaned_records
def normalize_responses(data, mapping):
    normalized_responses = []
    for row in data:
        normalized_row = []
        for word in row:
            normalized_word = mapping.get(word.lower())  # Default to "no respond" if not in mapping
            if normalized_word not in normalized_row:  # Avoid duplicates
                normalized_row.append(normalized_word)
        normalized_responses.append((set(normalized_row)))
        if not normalized_row:
            normalized_responses[-1] = ["no respond"]
    return normalized_responses

# Normalize responses using the updated mapping
normalized_responses4 = clean_none_values(normalize_responses(editedFourthColum, mapping))

# Print normalized responses
for i, response in enumerate(normalized_responses4):
    print(f"Record {i+1}: {response}")

Record 1: {'force myself to sleep'}
Record 2: {'listen to Quran'}
Record 3: ['no respond']
Record 4: ['no respond']
Record 5: {'force myself to sleep'}
Record 6: ['no respond']
Record 7: ['no respond']
Record 8: {'stay in bed'}
Record 9: {'listen to Quran', 'listen to podcast'}
Record 10: ['no respond']
Record 11: ['no respond']
Record 12: ['no respond']
Record 13: {'do other activities', 'listen to music', 'listen to podcast'}
Record 14: ['no respond']
Record 15: ['no respond']
Record 16: ['no respond']
Record 17: ['no respond']
Record 18: ['no respond']
Record 19: {'watch YouTube', 'drink water', 'watch videos'}
Record 20: ['no respond']
Record 21: ['no respond']
Record 22: ['no respond']
Record 23: ['no respond']
Record 24: {'listen to Quran', 'listen to podcast'}
Record 25: ['no respond']
Record 26: ['no respond']
Record 27: {'listen to music'}
Record 28: {'listen to Quran'}
Record 29: ['no respond']
Record 30: {'force myself to sleep'}
Record 31: ['no respond']
Record 32: {'clear 

In [29]:
FifthColum = translated_df.iloc[:,4].values

In [ ]:
FifthColum

In [ ]:
# Define a mapping dictionary for relevant actions or phrases
mapping = {
    "nothing": "do nothing",
    "no" : "do nothing",
    "drink": "drink water",
    "water": "drink water",
    "relax": "relax",
    "music": "listen to music",
    "quran": "listen to Quran",
    "read": "read",
    "story": "read a story",
    "exercise": "do exercise",
    "doctor": "consult a doctor",
    "breathe": "take a deep breath",
    "calm": "calm myself",
    "positive": "think positive thoughts",
    "write": "write to clear mind",
    "lighten": "lighten clothes",
    "friendly": "make bedroom sleep-friendly",
    "stress": "manage stress",
    "sit": "sit in bed",
    "overcome": "try to overcome",
    "sleep": "try to sleep",
    "close": "close eyes",
    "limon": "drink lemon water",
    "bed": "stay in bed",
    "wake": "wake up at the same time",
    "physical": "do physical activity",
    "manage": "manage stress",
    "computer": "avoid computer devices",
    "eyes": "rest eyes",
    "ved": "get rest",
}

# Function to extract relevant actions or phrases
def extract_relevant_phrases(column, mapping):
    extracted_phrases = []
    for sentence in column:
        relevant_phrases = []
        for word in str(sentence).split():  # Split sentence into words
            normalized_word = mapping.get(word.lower())
            if normalized_word:  # Check if the word is in the mapping
                relevant_phrases.append(normalized_word)
        if relevant_phrases:
            extracted_phrases.append((sorted(set(relevant_phrases))))  # Remove duplicates and join
        else:
            extracted_phrases.append("no response")  # If no relevant phrases found, add "no respond"
    return extracted_phrases



# Extract relevant phrases
extracted_phrases5 = extract_relevant_phrases(FifthColum, mapping)

# Print the extracted phrases
for i, phrase in enumerate(extracted_phrases5):
    print(f"Record {i+1}: {phrase}")

Record 1: no respond
Record 2: ['do nothing']
Record 3: no respond
Record 4: no respond
Record 5: no respond
Record 6: no respond
Record 7: no respond
Record 8: no respond
Record 9: no respond
Record 10: no respond
Record 11: no respond
Record 12: no respond
Record 13: no respond
Record 14: no respond
Record 15: no respond
Record 16: no respond
Record 17: ['do nothing']
Record 18: no respond
Record 19: no respond
Record 20: no respond
Record 21: no respond
Record 22: no respond
Record 23: no respond
Record 24: no respond
Record 25: no respond
Record 26: no respond
Record 27: no respond
Record 28: ['do nothing']
Record 29: no respond
Record 30: no respond
Record 31: no respond
Record 32: ['try to sleep']
Record 33: no respond
Record 34: no respond
Record 35: no respond
Record 36: ['do nothing']
Record 37: ['do nothing']
Record 38: ['do nothing']
Record 39: no respond
Record 40: ['drink water']
Record 41: no respond
Record 42: no respond
Record 43: no respond
Record 44: no respond
Record

In [32]:
sixthColum = translated_df.iloc[:,5].values


In [34]:

editedSixColum = []
for i in range(len(sixthColum)):
    row = []
    print(sixthColum[i])
    if pd.isna(sixthColum[i]) or not str(sixthColum[i]).strip() or str(sixthColum[i]).strip() == ".":
        row.append("no respnd")
    else:
        for token in nlp(sixthColum[i]):
            print(token.text, token.pos_, token.dep_)
            if token.pos_ == 'NUM':
                row.append(token.text)
        editedSixColum.append(row)
        print(row)
        print("--------------------------------------------------")

About 5 to 6
About ADV advmod
5 NUM quantmod
to PART quantmod
6 NUM ROOT
['5', '6']
--------------------------------------------------
into
into ADP ROOT
[]
--------------------------------------------------
into
into ADP ROOT
[]
--------------------------------------------------
More than 5 hours daily
More ADJ amod
than ADP quantmod
5 NUM nummod
hours NOUN npadvmod
daily ADV ROOT
['5']
--------------------------------------------------
9
9 NUM ROOT
['9']
--------------------------------------------------
into
into ADP ROOT
[]
--------------------------------------------------
During the day it can reach six hours, about an hour before bedtime
During ADP prep
the DET det
day NOUN pobj
it PRON nsubj
can AUX aux
reach VERB ROOT
six NUM nummod
hours NOUN dobj
, PUNCT punct
about ADV advmod
an PRON nummod
hour NOUN npadvmod
before ADP prep
bedtime NOUN pobj
['six']
--------------------------------------------------
4-5hours per day
4 NUM ROOT
- SYM punct
5hours NUM prep
per ADP prep
day N

In [35]:
from word2number import w2n

def convert_text_to_number(text):
    try:
        # Convert text to lowercase and use w2n to convert to a number
        return w2n.word_to_num(text.lower())
    except ValueError:
        # Return the original text if it cannot be converted
        return text
convertedSixColum = []
for row in editedSixColum:
    converted = [convert_text_to_number(example) for example in row]
    convertedSixColum.append(converted)
print(convertedSixColum)


[[5, 6], [], [], [5], [9], [], [6], [4, '5hours'], [5], [], [8], [], [], [], [], [], [], [], [18, 19], [8], [], [], [], [], [7], [], [3], [5], [], [], [], [], [3], ['0.5'], [6], [5], [], [3], [], [5], [], [9], [5, '+'], [], [6], [], [5], [7], [8], [6, 8], [13], [], [5, 8], [8], [], [], [2, 3], [], [], [8, 9], [15, 30], [], [8], [7], [], [6], [], [], [], [4, 5], [2, 3], [5], [4, 8], [], [3, 4], [8], [5], [], [], [], [8], [5], [], [5, 30], [4], [], [1], [9, 10], [], [], [], [], [], [], [16], [], [], [], [30], [12], ['2hours'], [], [], ['8hours'], [], [8, 10], [5, 6], [7], [5, 7], [2], [], [5], [], [4], [], ['1hour'], [5], [4], [], [4, 8], [], [], ['3or4'], [], [15], [], [1], [], [], [8], [7], [3, 5], [4], [4], [9, 12], [10], [12], [], [], [], [], [10], [8], [1], [10], [], [], ['2hours'], [1], [], [], [5], [5], [], [], [6, 8], [], [3], [4], [], [3, 4], [5], [5], [5], [5], [], [], [2, 3], [], [], [], [5, 6], [], [7, '8hours'], [1], [], [7], [], [], [9], [], [], [], [], [3, 4], [15], [], []

In [36]:
for i in range(len(convertedSixColum)):
    for j in range(len(convertedSixColum[i])):
        if isinstance(convertedSixColum[i][j], str):
            try:
                # Attempt to convert the string to an integer
                convertedSixColum[i][j] = int(convertedSixColum[i][j][0])
            except ValueError:
                # If conversion fails, leave the value as is
                convertedSixColum[i][j] = ''

In [37]:
numberList = []
for nums in convertedSixColum:
    n = []
    if not nums:
        continue
    for num in nums:
        if type(num) == int:
            n.append(num)
    total = sum(n)
    average = total / len(n)
    numberList.append(average)

In [38]:
print(numberList.__len__())

98


In [39]:
convertedSixColum.__len__()

193

In [74]:
age = df.iloc[:,[1]]

In [75]:
age.value_counts()

‎1. ‭What is your age ?
18 -- 25                   160
26 -- 35                    14
36 -- 45                    12
Above 46                     7
Under 18                     4
Name: count, dtype: int64

In [78]:
for i in range(len(age)):
    if convertedSixColum[i] == 0:
        if age.iloc[i,0] == "18 -- 25":
            convertedSixColum[i] = 5
        elif age.iloc[i,0] == "Under 18":
            convertedSixColum[i] = 1
        elif age.iloc[i,0] == "26 -- 35":
            convertedSixColum[i] = 1.5
        else:
            convertedSixColum[i] = 0.5

IndexError: list index out of range